In [16]:
# groupby("user").size()
import pandas as pd
import pickle
import numpy as np
import lenskit
import lenskit.crossfold as xf
from lenskit.algorithms.als import BiasedMF
from lenskit.algorithms.funksvd import FunkSVD
from lenskit.batch import predict
from lenskit.metrics.predict import rmse

In [17]:
ratings = pd.read_csv("../data/ml-1m/ratings.dat", sep="::", engine="python", header=None)

In [18]:
ratings = ratings.rename(columns={0: "user", 1: "item_id", 2: "rating", 3: "timestamp"})

In [19]:
# train20 = pd.read_pickle("../data/ml-20m-split/train.pkl")
# val20 = pd.read_pickle("../data/ml-20m-split/val.pkl")
# test20 = pd.read_pickle("../data/ml-20m-split/test.pkl")
full20 = pd.read_pickle("../data/ml-20m-split/full.pkl")
full20 = full20.rename(columns={'userId': 'user', 'movieId': 'item'})

In [20]:
full20

,user,item,item_id,rating,timestamp
0,0,1,2,3.5,1112486027
1,0,28,29,3.5,1112484676
2,0,31,32,3.5,1112484819
3,0,46,47,3.5,1112484727
4,0,49,50,3.5,1112484580
...,...,...,...,...,...
20000258,138492,13754,68954,4.5,1258126920
20000259,138492,13862,69526,4.5,1259865108
20000260,138492,13875,69644,3.0,1260209457
20000261,138492,13993,70286,5.0,1258126944


In [21]:
#ratings = pd.read_csv("../data/ml-20m/ratings.csv")
#ratings = ratings.rename(columns={"userId": "user", "movieId": "item_id"})
#ratings.groupby("item_id").size()
ratings

,user,item_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [22]:
item_map = ratings.groupby("item_id").size().reset_index()

In [23]:
keys = item_map["item_id"].values
values = np.arange(item_map.shape[0])
item_dict = dict([(k, v) for (k, v) in zip(keys, values)])

In [24]:
item_indices = [item_dict[item_id] for item_id in ratings["item_id"].values]

In [25]:
ratings["item"] = item_indices

In [26]:
column_titles = ["user", "item", "item_id", "rating", "timestamp"]
ratings = ratings.reindex(columns=column_titles)
ratings["user"] -= 1

In [27]:
ratings

,user,item,item_id,rating,timestamp
0,0,1104,1193,5,978300760
1,0,639,661,3,978302109
2,0,853,914,3,978301968
3,0,3177,3408,4,978300275
4,0,2162,2355,5,978824291
...,...,...,...,...,...
1000204,6039,1019,1091,1,956716541
1000205,6039,1022,1094,5,956704887
1000206,6039,548,562,5,956704746
1000207,6039,1024,1096,4,956715648


In [13]:
ratings = pd.read_pickle("../data/ml-20m-split/full.pkl")
ratings = ratings.rename(columns={'userId': 'user', 'movieId': 'item'})

In [28]:
# For train and test
split = xf.partition_users(ratings, 1, xf.SampleFrac(0.2))
train_test = next(split)
train = train_test.train
test = train_test.test

In [86]:
# For train and test (20M)
split20 = xf.partition_users(full20, 1, xf.SampleFrac(0.2))
train_test20 = next(split20)
train20 = train_test20.train
test20 = train_test20.test

In [103]:
# Toggle 20M
train = train20
test = test20

In [29]:
algo = FunkSVD(30)
print("fitting algo...")
algo.fit(train)
print("making predictions...")
preds = predict(algo, test)
overall_rmse = rmse(preds["prediction"], preds["rating"])
overall_rmse

fitting algo...
making predictions...


0.8612714514786182

In [52]:
algo.user_bias_

array([-0.40122879, -0.00467791,  0.32974058, ..., -0.14576175,
        0.15265135, -0.07985395])

In [31]:
algo.global_bias_

3.5817771462634638

In [53]:
algo.global_bias_ + algo.user_bias_[0] + algo.item_bias_[0] + np.dot(algo.user_features_[0], algo.item_features_[0])

2.792060423072559

In [49]:
preds

,user,item,item_id,rating,timestamp,prediction
40,0,0,1,5,978824268,4.298938
41,0,1781,1961,5,978301590,4.288115
36,0,1658,1836,5,978300172,3.669421
23,0,513,527,5,978824195,4.684207
10,0,581,595,5,978824268,4.190003
...,...,...,...,...,...,...
999890,6039,862,923,5,956716777,4.771588
1000157,6039,2202,2395,4,960972782,4.281869
1000125,6039,1224,1321,3,956715888,3.212127
999910,6039,917,978,4,957717557,3.926461


In [50]:
preds2

,user,item,item_id,rating,timestamp,prediction
40,0,0,1,5,978824268,4.307464
41,0,1781,1961,5,978301590,4.282101
36,0,1658,1836,5,978300172,3.383449
23,0,513,527,5,978824195,4.649099
10,0,581,595,5,978824268,4.328525
...,...,...,...,...,...,...
999890,6039,862,923,5,956716777,4.579140
1000157,6039,2202,2395,4,960972782,3.968331
1000125,6039,1224,1321,3,956715888,3.349932
999910,6039,917,978,4,957717557,4.258021


In [56]:
algo2.global_bias_ + algo2.user_bias_[0] + algo2.item_bias_[0] + np.dot(algo2.user_features_[0], algo2.item_features_[0])

4.4436142386519935

In [32]:
algo2 = BiasedMF(30, bias=True, iterations=100)
print("fitting algo...")
algo2.fit(train)
print("making predictions...")
preds2 = predict(algo2, test)
overall_rmse2 = rmse(preds2["prediction"], preds2["rating"])
overall_rmse2

fitting algo...
making predictions...


0.865832520582152

In [43]:
algo2.user_bias_

array([ 0.17082195, -0.08802569,  0.20192363, ..., -0.10820288,
       -0.11659151, -0.27317202])

In [37]:
algo2.global_bias_

3.5817771462634638

In [44]:
algo3 = BiasedMF(30, bias=False, iterations=100)
print("fitting algo...")
algo3.fit(train)
print("making predictions...")
preds3 = predict(algo3, test)
overall_rmse3 = rmse(preds3["prediction"], preds3["rating"])
overall_rmse3

fitting algo...
making predictions...


0.8571191488758345

In [47]:
algo3.user_bias_

In [48]:
algo3.global_bias_

0

In [28]:
# For train, validation, and test
split1 = xf.partition_users(ratings, 1, xf.SampleFrac(0.3))
train_valtest = next(split1)
train = train_valtest.train
valtest = train_valtest.test

In [29]:
# For train, validation, and test
split2 = xf.partition_users(valtest, 1, xf.SampleFrac(0.5))
val_test = next(split2)
val = val_test.train
test = val_test.test

In [13]:
train.groupby("user").size()

user
0        42
1       103
2        41
3        17
4       158
       ... 
6035    710
6036    162
6037     16
6038     98
6039    273
Length: 6040, dtype: int64

In [31]:
val.groupby("user").size()

user
0         8
1        19
2         7
3         3
4        29
       ... 
6035    133
6036     31
6037      3
6038     19
6039     51
Length: 6040, dtype: int64

In [14]:
test.groupby("user").size()

user
0        11
1        26
2        10
3         4
4        40
       ... 
6035    178
6036     40
6037      4
6038     25
6039     68
Length: 6040, dtype: int64

In [15]:
train.head()

,user,item,item_id,rating,timestamp
0,0,1104,1193,5,978300760
1,0,639,661,3,978302109
3,0,3177,3408,4,978300275
4,0,2162,2355,5,978824291
5,0,1107,1197,3,978302268


In [34]:
val.head()

,user,item,item_id,rating,timestamp
2,0,853,914,3,978301968
11,0,877,938,4,978301752
18,0,2889,3105,5,978301713
19,0,2592,2797,4,978302039
35,0,740,783,4,978824291


In [16]:
test.head()

,user,item,item_id,rating,timestamp
30,0,2102,2294,4,978824291
18,0,2889,3105,5,978301713
32,0,1439,1566,4,978824330
50,0,2898,3114,4,978302174
52,0,1154,1246,4,978302091


In [17]:
train.to_pickle("../data/ml-1m-split/train.pkl")
#val.to_pickle("../data/ml-1m-split/val.pkl")
test.to_pickle("../data/ml-1m-split/test.pkl")

In [18]:
ratings.to_pickle("../data/ml-1m-split/full.pkl")